# Building a Multi-Agent System with `smolagents`

In this notebook, we will explore how to use the [`smolagents`](https://huggingface.co/docs/smolagents) library from Hugging Face to build a simple multi-agent system.

Our goal is to create an agent system that collaborates to put together a lecture outline.  

## What You’ll Learn  

By the end of this exercise, you will:  
- Understand the basics of multi-agent systems and how agents can coordinate to solve tasks.  
- Learn how to define agents using `smolagents` and equip them with reasoning capabilities.  
- Explore how agents can use tools like web search to gather relevant information.  
- Implement a system where agents iteratively refine a lecture outline based on user prompts.  

## Getting Started  

Run the cells below to install dependencies and initialize the multi-agent system. Let's dive in!  


In [1]:
!pip install smolagents[openai]==1.11

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.9/89.9 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.3/105.3 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.1/13.1 MB 47.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 52.1 MB/s eta 0:00:00
  Attempting uninstall: pandas
    Found existing installation: pandas 2.2.2
    Uninstalling pandas-2.2.2:
      Successfully uninstalled pandas-2.2.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires pandas==2.2.2, but you have pandas 2.2.3 which is incompatible.


In [3]:
from smolagents import CodeAgent, DuckDuckGoSearchTool, OpenAIServerModel, LogLevel

## Setting Up Your OpenAI API Key  

This notebook requires an OpenAI API key to function. The key is used to authenticate requests when interacting with OpenAI's models.  

### How to Get an OpenAI API Key  

1. Go to the [OpenAI Platform](https://platform.openai.com/signup/) and sign up or log in.  
2. Navigate to the **API Keys** section in your account settings: [OpenAI API Keys](https://platform.openai.com/api-keys).  
3. Click **Create new secret key** and copy the generated key.  
4. Keep this key secure. Do not share it publicly or hardcode it in the notebook.  

Finally, execute the cell bellow and copy&paste the token into the provided field to configure the access of this notebook.

In [9]:
import os
from getpass import getpass

os.environ["OPENAI_API_KEY"] = getpass("Enter your OPENAI API key: ")

Enter your OPENAI API key: ··········


## Defining the LLM used by the models
In this notebook, our multi-agent system will rely on a Large Language Model (LLM) to generate and refine content.

The `smolagents` library allows us to define agents that interact with an LLM to reason, plan, and use external tools.  

We will use OpenAI's `gpt-4o` model. However, you can modify the setup to use other models.

If you prefer an open-source alternative, you can modify the notebook to use a Hugging Face-hosted model or a locally deployed LLM like LLaMA, Mistral, or Falcon.


In [6]:
model = OpenAIServerModel("gpt-4o", max_tokens=8096)

## Defining Expert Agents with `CodeAgent`  

In this section, we will define two specialized expert agents using `smolagents.CodeAgent`. Each agent will have a distinct area of expertise and will contribute to building a structured lecture outline.  

### What is `CodeAgent`?  

`CodeAgent` is a type of agent in the `smolagents` library that can generate and execute Python code as part of its reasoning process. This allows agents to perform computations, analyze data, and use external tools in a structured manner.  

### Defining the Experts  

We will create two expert agents:  

- **NLP Expert** – Specializes in natural language processing.  
- **Computer Vision Expert** – Focuses on computer vision tasks.  

Each agent will use an LLM to generate code-based responses when solving domain-specific tasks.

In [7]:
nlp_lecturer_agent = CodeAgent(
    name="nlp_lecturer",
    description="A lecturer in Natural Language Processing (NLP). This agent is a lecturer in NLP and can answer questions about NLP. When not sure about a topic, you can use the duckduckgo search tool to find the answer.",
    tools=[DuckDuckGoSearchTool()],
    model=model,
    max_steps=10
)

cv_lecturer_agent = CodeAgent(
    name="cv_lecturer",
    description="A lecturer in computer vision. This agent is a lecturer in CV and can answer questions about Computer Vision. When not sure about a topic, you can use the duckduckgo search tool to find the answer.",
    tools=[DuckDuckGoSearchTool()],
    model=model,
    max_steps=10
)


## Defining the Course Director Agent  

Now that we have our expert agents for NLP and Computer Vision, we need an agent to oversee the entire process. The **Course Director** agent will act as the central coordinator, ensuring that the experts collaborate effectively to construct a well-structured lecture outline.  

### Role of the Course Director  

The Course Director is responsible for:  
- **Managing the workflow** – Deciding when and how expert agents contribute.  
- **Gathering insights** – Requesting input from the NLP and Computer Vision experts.  
- **Refining the lecture outline** – Iteratively improving structure and content based on expert feedback.  
- **Ensuring coherence** – Making sure the final outline is clear, logical, and relevant.  

### Implementing the Course Director Agent  

We will define the Course Director as a `CodeAgent`, similar to the experts, but with a broader system prompt to guide decision-making and task delegation.


In [12]:
agent = CodeAgent(
    name='Course Director',
    description="A course director who manages lecturers.",
    tools=[],
    managed_agents=[cv_lecturer_agent, nlp_lecturer_agent],
    model=model,
    verbosity_level=LogLevel.DEBUG,
)


## Interacting with the Multi-Agent System  

Now that we have defined our agents—the **Course Director**, **NLP Expert**, and **Computer Vision Expert**—it's time to interact with them. You, as the user, will provide an initial topic or high-level prompt, and the agents will collaborate to generate a structured lecture outline.  


In [14]:

prompt = """
You are the course director of a university. You are responsible to coordinate the different topic experts and lecturers and consult them when ever neccessary to get the best possible answer. Always encurage the experts to do background research online prior answering the inquiry.
"Put me together an outline for a course on Generative AI"
"""
answer = agent.run(prompt)

╭─────────────────────────────────────────── New run - Course Director ───────────────────────────────────────────╮
│                                                                                                                 │
│ You are the course director of a university. You are responsible to coordinate the different topic experts and  │
│ lecturers and consult them when ever neccessary to get the best possible answer. Always encurage the experts to │
│ do background research online prior answering the inquiry.                                                      │
│ "Put me together an outline for a course on Generative AI"                                                      │
│                                                                                                                 │
╰─ OpenAIServerModel - gpt-4o ────────────────────────────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

Output message of the LLM: ────────────────────────────────────────────────────────────────────────────────────────
Understood! This task requires developing a comprehensive course outline on Generative AI. It would be prudent to  
involve the expert lecturers in NLP and Computer Vision, as these areas are pivotal in Generative AI. I will assign
tasks to each expert to contribute elements of the course they are familiar with, while encouraging them to conduct
online research for the most current and relevant information.                                                     
                                                                                                                   
Here are the steps I will take:                                                                                    
1. Engage the nlp_lecturer to provide an outline covering basic NLP concepts, deep learning techniques, and        
NLP-focused Generative AI models like GPT and BERT. I will encourage them to perform research to integrate the     
latest advancements in these areas.                                                                                
2. Engage the cv_lecturer to address Computer Vision applications of Generative AI, such as GANs, neural style     
transfer, and image-to-image translation. I will suggest they research current trends in these domains.            
3. Integrate both expert contributions into a holistic course outline with topics spanning both NLP and Computer   
Vision applications in Generative AI.                                                                              
                                                                                                                   
Let's proceed with the first step: engaging the nlp_lecturer.                                                      
                                                                                                                   
```py                                                                                                              
task_nlp_lecturer = """                                                                                            
You are tasked with developing an outline for a section of a Generative AI course focusing on Natural Language     
Processing (NLP).                                                                                                  
Please conduct an online research to ensure the following topics are up-to-date and relevant:                      
                                                                                                                   
[38;2;165;214;255;48;2

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  task_nlp_lecturer = """                                                                                          
  You are tasked with developing an outline for a section of a Generative AI course focusing on Natural Language   
  Processing (NLP).                                                                                                
  Please conduct an online research to ensure the following topics are up-to-date and relevant:                    
                                                                                                                   
  1. Introduction to Generative AI                                                                                 
  2. Fundamentals of Natural Language Processing                                                                   
  3. Introduction to Deep Learning concepts and frameworks                                                         
  4. Generative Models in NLP - Overview of GPT series, BERT, and other transformational models                    
  5. Advanced Topics: Few-shot learning, prompt engineering                                                        
  6. Ethical Considerations and AI Bias in NLP                                                                     
                                                                                                                   
  Please provide a detailed outline with subtopics for each of these areas, considering both foundational          
  knowledge and cutting-edge research and techniques.                                                              
  """                                                                                                              
                                                                                                                   
  response_nlp_lecturer = nlp_lecturer(task=task_nlp_lecturer)                                                     
  print(response_nlp_lecturer)                                                                                     
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

╭──────────────────────────────────────────── New run - nlp_lecturer ─────────────────────────────────────────────╮
│                                                                                                                 │
│ You're a helpful agent named 'nlp_lecturer'.                                                                    │
│ You have been submitted this task by your manager.                                                              │
│ ---                                                                                                             │
│ Task:                                                                                                           │
│                                                                                                                 │
│ You are tasked with developing an outline for a section of a Generative AI course focusing on Natural Language  │
│ Processing (NLP).                                                                                               │
│ Please conduct an online research to ensure the following topics are up-to-date and relevant:                   │
│                                                                                                                 │
│ 1. Introduction to Generative AI                                                                                │
│ 2. Fundamentals of Natural Language Processing                                                                  │
│ 3. Introduction to Deep Learning concepts and frameworks                                                        │
│ 4. Generative Models in NLP - Overview of GPT series, BERT, and other transformational models                   │
│ 5. Advanced Topics: Few-shot learning, prompt engineering                                                       │
│ 6. Ethical Considerations and AI Bias in NLP                                                                    │
│                                                                                                                 │
│ Please provide a detailed outline with subtopics for each of these areas, considering both foundational         │
│ knowledge and cutting-edge research and techniques.                                                             │
│                                                                                                                 │
│ ---                                                                                                             │
│ You're helping your manager solve a wider task: so make sure to not provide a one-line answer, but give as much │
│ information as possible to give them a clear understanding of the answer.                                       │
│                                                                                                                 │
│ Your final_answer WILL HAVE to contain these parts:                                                             │
│ ### 1. Task outcome (short version):                                                                            │
│ ### 2. Task outcome (extremely detailed version):                                                               │
│ ### 3. Additional context (if relevant):                                                                        │
│                                                                                                                 │
│ Put all these in your final_answer tool, everything that you do not pass as an argument to final_answer will be │
│ lost.                                                                                                           │
│ And even if your task resolution is not successful, please return as much context as possible, so that your     │
│ manager can act upon this feedback.                                                                             │
│                                                       

...

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 4 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

Output message of the LLM: ────────────────────────────────────────────────────────────────────────────────────────
We now have a complete and detailed course outline for a Generative AI course, which includes both Natural Language
Processing and Computer Vision sections. All sections have been enriched with up-to-date insights, advanced topics,
and ethical considerations.                                                                                        
                                                                                                                   
Here is a summary of the course outline:                                                                           
                                                                                                                   
### Course on Generative AI                                                                                        
                                                                                                                   
#### Section 1: Natural Language Processing (NLP)                                                                  
1. **Introduction to Generative AI**                                                                               
   - Definition, current applications, technological foundations, future prospects.                                
2. **Fundamentals of Natural Language Processing**                                                                 
   - Core concepts, modern applications, and future directions.                                                    
3. **Deep Learning Concepts and Frameworks**                                                                       
   - Overview of neural network architectures and frameworks like TensorFlow and PyTorch.                          
4. **Generative Models in NLP**                                                                                    
   - Overview of models like BERT, GPT series.                                                                     
5. **Advanced Topics**                                                                                             
   - Few-shot learning and prompt engineering.                                                                     
6. **Ethical Considerations and AI Bias**                                                                          
   - Analyzing potential biases, regulatory and social considerations.                                             
                                                                                                                   
#### Section 2: Computer Vision (CV)                                                                               
1. **Introduction to Generative AI in Computer Vision**                                                            
   - Historical context, technological advancements, industry applications, and future directions.                 
2. **Basics of Computer Vision**                                                                                   
   - Foundational concepts like image processing, core techniques, and probabilistic approaches.                   
3. **Deep Learning in Computer Vision**                                                                            
   - Core techniques and advancements in CNN architectures.                                                        
4. **Generative Models for CV**                                                                                    
   - GANs, VAEs, and recent innovations.                                                                           
5. **Applications**                                                                                                
   - Neural style transfer, image-to-image translation.                                                            
6. **Ethical Consideration

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  final_answer({                                                                                                   
      "Course Outline": {                                                                                          
          "NLP Section": {                                                                                         
              "1. Introduction to Generative AI": [                                                                
                  "Definition",                                                                                    
                  "Current Applications",                                                                          
                  "Technological Foundations",                                                                     
                  "Future Prospects"                                                                               
              ],                                                                                                   
              "2. Fundamentals of Natural Language Processing": [                                                  
                  "Core Concepts",                                                                                 
                  "Modern Applications",                                                                           
                  "Future Directions"                                                                              
              ],                                                                                                   
              "3. Deep Learning Concepts and Frameworks": [                                                        
                  "Neural Network Architectures",                                                                  
                  "Frameworks like TensorFlow and PyTorch"                                                         
              ],                                                                                                   
              "4. Generative Models in NLP": [                                                                     
                  "BERT",                                                                                          
                  "GPT Series Models"                                                                              
              ],                                                                                                   
              "5. Advanced Topics": [                                                                              
                  "Few-shot Learning",                                                                             
                  "Prompt Engineering"                                                                             
              ],                                                                                                   
              "6. Ethical Considerations and AI Bias": [                                                           
                  "Analyzing Potential Biases",                                                                    
                  "Regulatory and Social Considerations"                                                           
              ]                                                                                                    
          },                                                                                                       
          "CV Section": {                                                                                          
              "1. Introduction to Generative AI in Computer Vision": [                                             
                  "Historical Context",                  

Out - Final answer: {'Course Outline': {'NLP Section': {'1. Introduction to Generative AI': ['Definition', 'Current
Applications', 'Technological Foundations', 'Future Prospects'], '2. Fundamentals of Natural Language Processing': 
['Core Concepts', 'Modern Applications', 'Future Directions'], '3. Deep Learning Concepts and Frameworks': ['Neural
Network Architectures', 'Frameworks like TensorFlow and PyTorch'], '4. Generative Models in NLP': ['BERT', 'GPT 
Series Models'], '5. Advanced Topics': ['Few-shot Learning', 'Prompt Engineering'], '6. Ethical Considerations and 
AI Bias': ['Analyzing Potential Biases', 'Regulatory and Social Considerations']}, 'CV Section': {'1. Introduction 
to Generative AI in Computer Vision': ['Historical Context', 'Technological Advancements', 'Industry Applications',
'Future Directions'], '2. Basics of Computer Vision': ['Image Processing', 'Core Techniques', 'Probabilistic 
Approaches'], '3. Deep Learning in Computer Vision': ['CNN Architectures', 'Recent Advancements'], '4. Generative 
Models for CV': ['GANs', 'VAEs', 'Recent Innovations'], '5. Applications': ['Neural Style Transfer', 
'Image-to-Image Translation'], '6. Ethical Considerations and Challenges': ['Addressing Biases', 'Privacy and 
Surveillance Impacts', 'Security Concerns', 'Regulatory Frameworks']}}}

[Step 4: Duration 13.52 seconds| Input tokens: 33,130 | Output tokens: 2,518]

In [17]:
print(answer)

{'Course Outline': {'NLP Section': {'1. Introduction to Generative AI': ['Definition', 'Current Applications', 'Technological Foundations', 'Future Prospects'], '2. Fundamentals of Natural Language Processing': ['Core Concepts', 'Modern Applications', 'Future Directions'], '3. Deep Learning Concepts and Frameworks': ['Neural Network Architectures', 'Frameworks like TensorFlow and PyTorch'], '4. Generative Models in NLP': ['BERT', 'GPT Series Models'], '5. Advanced Topics': ['Few-shot Learning', 'Prompt Engineering'], '6. Ethical Considerations and AI Bias': ['Analyzing Potential Biases', 'Regulatory and Social Considerations']}, 'CV Section': {'1. Introduction to Generative AI in Computer Vision': ['Historical Context', 'Technological Advancements', 'Industry Applications', 'Future Directions'], '2. Basics of Computer Vision': ['Image Processing', 'Core Techniques', 'Probabilistic Approaches'], '3. Deep Learning in Computer Vision': ['CNN Architectures', 'Recent Advancements'], '4. Gen

## Congratulations!

You have successfully completed this exercise on building a multi-agent system using `smolagents`. Throughout this notebook, you:  

- Learned how to define and configure expert agents using `CodeAgent`.  
- Created an **NLP Expert** and a **Computer Vision Expert** to provide domain-specific insights.  
- Implemented a **Course Director** agent to coordinate the workflow and manage agent interactions.  
- Interacted with the multi-agent system to generate a structured lecture outline.  
- Observed how agents collaborate, exchange information, and iteratively refine their output.  

This exercise demonstrated how multi-agent systems can be used to automate complex reasoning and content creation tasks. You can extend this framework by adding new expert agents, integrating additional tools, or refining the agents' prompts to improve performance.  
